In [1]:
import pandas as pd
import numpy as np
import csv
import glob
import os

In [2]:
samsung_stoke_data_base = r'D:\dataset\stoke\samsung\data'
samsung_stoke_label_base = r'D:\dataset\stoke\samsung\label'
oil_data_base = r'D:\dataset\stoke\oil\data'
oil_label_base = r'D:\dataset\stoke\oil\label'
ex_bio_data_base = r'D:\dataset\stoke\ex_bio\data'
ex_bio_label_base = r'D:\dataset\stoke\ex_bio\label'
usecols = ['time', 'price', 'sell', 'buy', 'trading_volum']

## naver에서 data를 csv파일로 가져옴
   - chrome에서 f12눌러서 확인

In [137]:
#/item/sise_time.nhn?code=005930&thistime=20201015134741 삼성
#http://finance.naver.com/item/sise_time.nhn?code=950130&thistime=20201014144608 ex_bio
# /item/sise_time.nhn?code=500019&thistime=20201027123806  oil

In [3]:
url='http://finance.naver.com/item/sise_time.nhn?code=950130&thistime=20201102134741'
df=pd.DataFrame()
for page in range(1, 41):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

In [4]:
df.rename(columns={'체결시각': 'time',
                   '체결가': 'price',
                   '전일비' : 'trash',
                  '매도': 'sell',
                  '매수': 'buy',
                  '거래량': 'trading_volum',
                  '변동량': 'trading_validation'},
          
                   inplace=True)

##  데이터 추출
   
   - fillna(0)를 통해 nan값을 0으로 채워줌
   
   - 그 후 drop을 통해 count_list에 있는 숫자(순서)의 데이터들을 뺌


In [5]:
df['price'] = df['price'].fillna(0)

In [6]:
count = 1
count_list = []

for k in df['price']:
    if k == 0:
        count_list.append(count)
    count += 1

new_count = np.array(count_list)

data_df = df
data_df.drop(data_df.index[new_count - 1], inplace=True)
print('ok')

ok


- # data가 시간순서의 역행하는 방행으로 있기 때문에 뒤집어줌

In [7]:
data_df = data_df.loc[::-1]

In [8]:
data_df

,time,price,trash,sell,buy,trading_volum,trading_validation
596,09:00,29000.0,400.0,29050.0,29000.0,12408.0,12408.0
595,09:01,28850.0,250.0,28900.0,28850.0,25364.0,12956.0
594,09:02,28950.0,350.0,28950.0,28900.0,34927.0,9563.0
590,09:03,28950.0,350.0,28950.0,28900.0,39859.0,4932.0
589,09:04,28850.0,250.0,28900.0,28850.0,46778.0,6919.0
...,...,...,...,...,...,...,...
5,13:43,29250.0,650.0,29300.0,29250.0,666156.0,1552.0
4,13:44,29300.0,700.0,29300.0,29250.0,666286.0,130.0
3,13:45,29300.0,700.0,29300.0,29250.0,666379.0,93.0
2,13:46,29300.0,700.0,29300.0,29250.0,669129.0,2750.0


- # csv_data 저장

In [9]:
data_df.to_csv(ex_bio_data_base + '\\20201102.csv', index=False)

## label 만들기

    - data중 price를 불러옴
    
    - 이전 데이터와 비교해서 price가 늘었나 줄었나 같나 확인
    
    - 같음: 0 , 늚: 1, 줄음: 2 로 원핫인코딩 할 예정

In [10]:
data_csv= open(ex_bio_data_base + '\\20201102.csv', 'r')
data = csv.reader(data_csv)

In [11]:
load_data_df = pd.DataFrame(data) 

In [12]:
load_data_df

,0,1,2,3,4,5,6
0,time,price,trash,sell,buy,trading_volum,trading_validation
1,09:00,29000.0,400.0,29050.0,29000.0,12408.0,12408.0
2,09:01,28850.0,250.0,28900.0,28850.0,25364.0,12956.0
3,09:02,28950.0,350.0,28950.0,28900.0,34927.0,9563.0
4,09:03,28950.0,350.0,28950.0,28900.0,39859.0,4932.0
...,...,...,...,...,...,...,...
372,13:43,29250.0,650.0,29300.0,29250.0,666156.0,1552.0
373,13:44,29300.0,700.0,29300.0,29250.0,666286.0,130.0
374,13:45,29300.0,700.0,29300.0,29250.0,666379.0,93.0
375,13:46,29300.0,700.0,29300.0,29250.0,669129.0,2750.0


In [13]:
load_data_df[1][1:]

1      29000.0
2      28850.0
3      28950.0
4      28950.0
5      28850.0
        ...   
372    29250.0
373    29300.0
374    29300.0
375    29300.0
376    29250.0
Name: 1, Length: 376, dtype: object

- # data를 int형식으로 바꿔 리스트에 넣어줌

In [14]:
tmp_label = []

for i in load_data_df[1][1:]:
    int_label = float(i)
    tmp_label.append(int_label)
    
del int_label

In [15]:
type(tmp_label[0])

float

In [16]:
labels = []

for i in range(1, len(tmp_label)):
    if (tmp_label[i-1] - tmp_label[i]) > 0:
        labels.append(2)
    elif (tmp_label[i-1] - tmp_label[i]) == 0:
        labels.append(0)
    else:
        labels.append(1)

In [17]:
labels = np.array(labels)

In [18]:
labels

array([2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1,
       1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2,
       0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1,
       0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2,
       2, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2, 1, 1, 2, 0, 1, 1, 1,
       2, 1, 2, 0, 2, 1, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 1, 1, 1, 0, 2,
       0, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0,
       2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 0, 2,
       0, 2, 1, 0, 1, 1, 0, 1, 1, 2, 2, 1, 0, 2, 1, 2, 0, 2, 0, 0, 1, 0,
       0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0,
       1, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 2, 1,
       0, 2, 0, 0, 0, 0, 1, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 0, 1, 1, 2, 0,
       0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 1,
       1, 2, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 1, 2, 0,

- # label을 numpy로 저장해줌

In [19]:
np.save(ex_bio_label_base + '\\20201102.npy', labels)

## 데이터를 받는 class 만듦

   - value1: data_path
   - value2: label_path
   - value3: url

In [5]:
class Data_harvasting:
    data_path = ''
    label_path = ''
    url = ''
    date = ''
    code = ''
    
    def __init__(self, value1, value2, value3, value4):
        self.data_path = value1
        self.label_path = value2
        self.code = value3
        self.date = value4
        self.url = 'http://finance.naver.com/item/sise_time.nhn?code={code}&thistime={date}'.format(code = self.code, date = self.date)
     
    #url에서 데이터를 받음
    def harvasting_data(self):
        
        #url에서 데이터를 가져옴
        df=pd.DataFrame()
        for page in range(1, 41):
            pg_url = '{url}&page={page}'.format(url=self.url, page=page)
            df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
            
        df.rename(columns={'체결시각': 'time',
                   '체결가': 'price',
                   '전일비' : 'trash',
                  '매도': 'sell',
                  '매수': 'buy',
                  '거래량': 'trading_volum',
                  '변동량': 'trading_validation'},
          
                   inplace=True)
        
        #비어있는 값은 없에기
        df['price'] = df['price'].fillna(0)
        count = 1
        count_list = []
        
        for k in df['price']:
            if k == 0:
                count_list.append(count)
            count += 1
            
        new_count = np.array(count_list)
        data_df = df
        data_df.drop(data_df.index[new_count - 1], inplace=True)
        print('ok')
        
        #데이터 시간을 반대로 뒤집어줌
        data_df = data_df.loc[::-1]
        
        #csv데이터 저장
        data_df.to_csv(self.data_path + '\\{date}.csv'.format(date = self.date[0:8]), index=False)
     
    #label을 생성함
    def harvasting_label(self):
        data_csv= open(self.data_path + '\\{date}.csv'.format(date = self.date[0:8]), 'r')
        data = csv.reader(data_csv)
        load_data_df = pd.DataFrame(data) 
        
        tmp_label = []
        for i in load_data_df[1][1:]:
            int_label = float(i)
            tmp_label.append(int_label)
        del int_label
        
        labels = []
        for i in range(1, len(tmp_label)):
            if (tmp_label[i-1] - tmp_label[i]) > 0:
                labels.append(2)
            elif (tmp_label[i-1] - tmp_label[i]) == 0:
                labels.append(0)
            else:
                labels.append(1)
        
        labels = np.array(labels)
        np.save(self.label_path + '\\{date}.npy'.format(date = self.date[0:8]), labels)

# code랑 date 미리 적어두기

   - oil: 500019,    date: 20201027123806
   
   - samsung: 005930,    date: 20201015134741
   
   - ex_bio: 950130,     date: 20201014144608

In [11]:
code_name = '950130'
data_base = ex_bio_data_base
label_base = ex_bio_label_base

In [48]:
test1 = Data_harvasting(data_base, label_base, code_name, '20201103134741')
test1.harvasting_data()
test1.harvasting_label()

test2 = Data_harvasting(data_base, label_base, code_name, '20201104134741')
test2.harvasting_data()
test2.harvasting_label()

test3 = Data_harvasting(data_base, label_base, code_name, '20201105134741')
test3.harvasting_data()
test3.harvasting_label()

test4 = Data_harvasting(data_base, label_base, code_name, '20201106134741')
test4.harvasting_data()
test4.harvasting_label()

test5 = Data_harvasting(data_base, label_base, code_name, '20201109134741')
test5.harvasting_data()
test5.harvasting_label()


ok
ok
ok
ok
ok


In [13]:
test1 = Data_harvasting(data_base, label_base, code_name, '20201112144608')
test1.harvasting_data()
test1.harvasting_label()

ok
